# Example 1
This notebook's cells will run a small-scale simulation of methdemon for a tumour evolving via gland fission and visualise the methylation arrays of the glands.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
# Define relative paths to output files
outputs_path = 'outputs/'
methylation_file = 'output_methylation_arrays.dat'
demes_file = 'output_deme_methylation.dat'

In [ ]:
# Uncomment the below three lines and run if methdemon has not been compiled yet
# NOTE: Change the path to the include directory of your boost installation

#!cd ../../src
#!g++ -o methdemon methdemon.cpp -I/usr/local/include/ -lm  
#!cd ../examples/eg1

In [ ]:
# Run the example simulation
!../../src/methdemon ./outputs configfile-eg1.dat

In [ ]:
# Read in the output files
methylation = pd.read_csv(outputs_path + methylation_file, sep='\t')
demes = pd.read_csv(outputs_path + demes_file, sep='\t')
carrying_capacity = demes.Population.max()
filled_demes = demes[demes['Population'] == carrying_capacity]
filled_demes.head()

In [ ]:
# Separate the demes methylation array strings into lists of floats
deme_arrays = filled_demes.AverageArray.apply(lambda x: [float(i) for i in x.split()])

In [ ]:
# Plot a randomly selected deme's methylation array
# Randomly select deme to plot
deme = np.random.randint(0, len(deme_arrays))
# Plot the deme's methylation histogram
plt.hist(deme_arrays.iloc[deme], bins=50)
plt.title('Deme ' + str(deme) + ' methylation array')

In [ ]:
# Set origin coordinates in the middle of the tumour
xorigin = filled_demes.X.median()
yorigin = filled_demes.Y.median()

In [ ]:
# Separate tumour into left and right sides
left_glands = filled_demes[filled_demes['X'] < xorigin].sort_values(by='Y')
right_glands = filled_demes[filled_demes['X'] >= yorigin].sort_values(by='Y')

In [ ]:
def select4(glands):
    mid = len(glands) // 2
    if len(glands) % 2 == 0:
        return pd.concat([glands.iloc[mid-2 : mid], glands.iloc[mid : mid+2]])
    else:
        return pd.concat([glands.iloc[mid-2 : mid], glands.iloc[mid+1 : mid+3]])

In [ ]:
left_selection = select4(left_glands)
right_selection = select4(right_glands)

In [ ]:
left_arrays = left_selection.AverageArray.apply(lambda x: [float(i) for i in x.split()])
right_arrays = right_selection.AverageArray.apply(lambda x: [float(i) for i in x.split()])

In [ ]:
# Plot the methylation arrays of the selected demes on the same histogram with left glands being in a blue colour palette and right glands in orange
plt.figure(figsize=(10, 5))
for i in range(len(left_arrays)):
    plt.hist(left_arrays.iloc[i], bins=50, color='blue', alpha=0.5)
    plt.hist(right_arrays.iloc[i], bins=50, color='orange', alpha=0.5)

In [ ]:
left_arrays = left_arrays.reset_index(drop=True)
right_arrays = right_arrays.reset_index(drop=True)
all_samples = pd.concat([left_arrays, right_arrays], axis=0)

In [ ]:
# Correlation plots for each two sampled demes

num_samples = 8
fig, axes = plt.subplots(num_samples, num_samples, figsize=(15, 15))

for i in range(num_samples):
    for j in range(num_samples):
        if j > i:  # We only plot above the diagonal
            sns.scatterplot(data=all_samples, x=all_samples.iloc[i], y=all_samples.iloc[j], ax=axes[i, j])
        else:
            axes[i, j].set_axis_off()  # Turn off axes for other plots

plt.tight_layout()
plt.show()